In [2]:
import json
import pandas as pd
import requests

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_columns', 500)

## California

In [9]:
df_pcp = pd.read_csv('../data/private/PCPs FTE 2015.csv')
df_spe = pd.read_csv('../data/private/Specialists FTE 2015.csv')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
df_pcp.head(5)

,ID,Plan ID,Health Plan,Reporting for Health Plan,Network Name,NPI,Num Network Enrollees,Num Network Specialists,Network Expected Enrollees to Specs,Total Expected Enrollment,Min Regional Expected Enrollment,Max Regional Expected Enrollment,Regions,Provider Expected Enrollment,FTE
0,1,933 0043,California Physicians' Service,NaN,Access+ HMO,900000048,529267,27126,19.511428,37.289217,23.199474,23.199474,1,23.199474,0.071539
1,2,933 0043,California Physicians' Service,NaN,Access+ HMO,900000061,529267,27126,19.511428,37.289217,23.199474,23.199474,1,23.199474,0.071539
2,3,933 0043,California Physicians' Service,NaN,Access+ HMO,900000072,529267,27126,19.511428,18.162223,12.244241,12.244241,1,12.244241,0.040124
3,4,933 0043,California Physicians' Service,NaN,Access+ HMO,900000086,529267,27126,19.511428,93.313054,5.715756,5.715756,1,5.715756,0.015029
4,5,933 0043,California Physicians' Service,NaN,Access+ HMO,900000089,529267,27126,19.511428,75.102483,14.842240,14.842240,1,14.842240,0.040989


## Find Largest Networks and Plans
### PCPs

In [65]:
df_networks = df_pcp.groupby(['Health Plan', 'Networkname'])['Num Network Enrollees'].max()
df_plans = df_networks.groupby(['Health Plan']).sum()

In [66]:
df_networks = df_networks.sort_values(ascending=False)
df_plans = df_plans.sort_values(ascending=False)

In [67]:
# Save to CSV
df_networks.to_csv('../data/private/PCP_networks_2015.csv')
df_plans.to_csv('../data/Private/PCP_plans_2015.csv')

### Specialists

In [68]:
df_networks = df_spe.groupby(['Health Plan', 'Network Name'])['Num Network Enrollees'].max()
df_plans = df_networks.groupby(['Health Plan']).sum()

In [69]:
df_networks = df_networks.sort_values(ascending=False)
df_plans = df_plans.sort_values(ascending=False)

# Note - results are identical: good news.

### List specialties 

In [ ]:
# Loop through Specialist2016 California data to list available specialties.
chunksize = 10 ** 4
specialties = set()
try:
    for chunk in pd.read_csv('../data/private/IndividualSB964Specialist2016.csv', chunksize=chunksize, encoding="ISO-8859-1"):
        temp_specialties = chunk['SpecialistSpecialty'].unique()
        specialties.update(temp_specialties)
except:
    pass

In [1]:
# Print list.
sorted(list(specialties))

NameError: name 'specialties' is not defined

## Texas

In [7]:
plan_index = [
    'https://d3ul0st9g52g6o.cloudfront.net/ffm/cms-data-index.json',
    # 'https://www.dentegra.com/hcx/cms-data-index.json',
    # 'http://mydental.guardianlife.com/secure/json/index.json',
    'https://my.firstcare.com/CMS/MachineReadableFormat/cms-data-index.json',
    'https://providersearch.communitycares.com/webservice/json/display/him/index.json',
    'https://api.centene.com/ambetter/reference/cms-data-index.json',
    'https://www.bcbstx.com/forms/tx/index_tx.JSON',
    'https://cmsdata.providers4you.com/cms-data-index.json',
    'http://stg-oh-medicaid.molinahealthcare.com/json/index.json',
    'http://www.bestlife.com/exchange/cms-data-index.json',
    # 'https://www.dentaquest.com/getattachment/Marketplace/CMSDataIndex/cms-data-index.json',
    'https://api.humana.com/v1/cms/index.json',
    'https://chppayment.christushealth.org/workfiles/json/index.json',
    'https://senderohealth.com/idealcareeng/json/jsonfiles/index.json',
    'https://emihealth.com/services/providerdata/cms-data-index.json'
]

# plan_index = [
#     'https://my.firstcare.com/CMS/MachineReadableFormat/cms-data-index.json',
#     'https://providersearch.communitycares.com/webservice/json/display/him/index.json',
#     'https://cmsdata.providers4you.com/cms-data-index.json'
# ]

In [8]:
# Find provider urls for each plan.
provider_urls = []
for url in plan_index:
    try:
        response = requests.get(url)
        response.raise_for_status()
        response_json = response.json()
        # print(response_json)
        if 'provider_urls' in response_json:
            provider_urls.extend(response_json['provider_urls'])
        print('Processed {}'.format(url))
    except:
        print('Error parsing {}'.format(url))
        continue

Processed https://d3ul0st9g52g6o.cloudfront.net/ffm/cms-data-index.json
Error parsing https://my.firstcare.com/CMS/MachineReadableFormat/cms-data-index.json
Error parsing https://providersearch.communitycares.com/webservice/json/display/him/index.json
Processed https://api.centene.com/ambetter/reference/cms-data-index.json
Processed https://www.bcbstx.com/forms/tx/index_tx.JSON
Error parsing https://cmsdata.providers4you.com/cms-data-index.json
Processed http://stg-oh-medicaid.molinahealthcare.com/json/index.json
Processed http://www.bestlife.com/exchange/cms-data-index.json
Processed https://api.humana.com/v1/cms/index.json
Processed https://chppayment.christushealth.org/workfiles/json/index.json
Processed https://senderohealth.com/idealcareeng/json/jsonfiles/index.json
Processed https://emihealth.com/services/providerdata/cms-data-index.json


In [9]:
df = pd.DataFrame(provider_urls, columns=["urls"])
df.to_csv('../data/private/provider_urls.csv', index=False)

In [4]:
provider_urls_df = pd.read_csv('../data/private/provider_urls.csv')
provider_urls = provider_urls_df['urls'].tolist()

In [ ]:
# Extract provider info for each plan.
providers = []
for provider_url in provider_urls:
    print(provider_url)
    try:
        response = requests.get(provider_url)
        response.raise_for_status()
        response_json = response.json()
        providers.extend(response_json)
        print('Processed {}'.format(provider_url))
    except:
        print('Error fetching {}'.format(provider_url))
        continue

https://d3ul0st9g52g6o.cloudfront.net/ffm/providers.json
Processed https://d3ul0st9g52g6o.cloudfront.net/ffm/providers.json
https://api.centene.com/ambetter/reference/provider_facility.json
Processed https://api.centene.com/ambetter/reference/provider_facility.json
https://api.centene.com/ambetter/reference/provider_individual.json
Processed https://api.centene.com/ambetter/reference/provider_individual.json
https://www.bcbstx.com/forms/tx/medical_provider_tx-1.json


In [3]:
# Save providers to file.
json.dump(providers, '../data/private/providers.json')

NameError: name 'providers' is not defined

In [ ]:
# Find plans with most providers
plans = defaultdict(set)
for provider in providers:
    for plan in provider['plans']:
        
{"npi":"1013236256","type":"INDIVIDUAL","name":{"prefix":"Dr.","first":"Alicia","middle":"L","last":"Johnson"},"addresses":[{"address":"17510 W Grand Pkwy S","address_2":"Ste 530","city":"Sugar Land","state":"TX","zip":"77479","phone":"2813130090"},{"address":"7737 Southwest Fwy","address_2":"Ste 500","city":"Houston","state":"TX","zip":"77074","phone":"2813130090"}],"specialty":["Podiatry"],"accepting":"accepting","gender":"Female","plans":[{"plan_id_type":"HIOS-PLAN-ID","plan_id":"32673TX0630002","network_tier":"PREFERRED","years":[2017]},{"plan_id_type":"HIOS-PLAN-ID","plan_id":"32673TX0630003","network_tier":"PREFERRED","years":[2017]},{"plan_id_type":"HIOS-PLAN-ID","plan_id":"32673TX0630004","network_tier":"PREFERRED","years":[2017]}